In [5]:
import streamlit as st
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import  mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np 
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
from cato import cato_1
from lable import label_1
from outlier import out_1
from min_max import mima
from data_cleaning import cleaning


In [20]:
dx = pd.read_csv('C:/Users/Hp/Downloads/playground-series-s4e12/train.csv')
dz = pd.read_csv('C:/Users/Hp/Downloads/playground-series-s4e12/test.csv')
data_1 = dz.head(1).iloc[:,[1,2,3,5,8,9,11,13,14,17]]
data_1['Previous Claims'] = data_1['Previous Claims'].fillna(5)
data_1['Credit Score'] = data_1['Credit Score'].fillna(590)
data_1

,Age,Gender,Annual Income,Number of Dependents,Health Score,Location,Previous Claims,Credit Score,Insurance Duration,Smoking Status
0,28.0,Female,2310.0,4.0,7.657981,Rural,5.0,590.0,1.0,Yes


In [21]:
data_1

,Age,Gender,Annual Income,Number of Dependents,Health Score,Location,Previous Claims,Credit Score,Insurance Duration,Smoking Status
0,28.0,Female,2310.0,4.0,7.657981,Rural,5.0,590.0,1.0,Yes


In [22]:
train_data_1 = cleaning(dx)
train_data = train_data_1.data_cleaning()


cato_3 = train_data

categorical_columns = ['Gender','Smoking Status']
        
cato_train_99 = cato_1(cato_3,categorical_columns)
cato_train = cato_train_99.categorical()

    
cato_train_101 = label_1(cato_train)
cato_train_1 = cato_train_101.label()

    
train_out_90 = out_1(cato_train_1)
train_out = train_out_90.Find_outlier()

    
categorical_columns_cato_train_2=[]
for cato_train_1_1 in train_out:
           categorical_columns_cato_train_2.append(cato_train_1_1)
            
cato_train_38 = mima(cato_train_1,categorical_columns_cato_train_2)
cato_train_3 = cato_train_38.min_max()

A = cato_train_3.drop('Premium Amount', axis=1)
b = cato_train_3['Premium Amount']
A = A[sorted(A.columns)]

test_data = data_1

cato_test_10  = cato_1(data_1,categorical_columns)
cato_test = cato_test_10.categorical()

    
cato_test_101  = label_1(cato_test)
cato_test_1 = cato_test_101.label()

In [31]:
cato_test_1.dtypes

Age                                 float64
Annual Income                       float64
Credit Score                        float64
Gender_Female                       float64
Gender_Male                         float64
Health Score                        float64
Insurance Duration                  float64
Location_encoder                     object
Number of Dependents                float64
Previous Claims                     float64
Smoking Status_No                   float64
Smoking Status_Yes                  float64
Predicted_Target_premium_account    float32
Gender                               object
Smoking Status                       object
dtype: object

In [24]:
A_train, A_test, b_train, b_test = train_test_split(A, b, test_size = 0.20 , random_state = 42)

XGR_Boost = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5)
XGR_Boost.fit(A_train,b_train)

B_pred = XGR_Boost.predict(A_test)
signature = infer_signature(A_train, XGR_Boost.predict(A_train))

c:\Users\Hp\OneDrive\Desktop\python\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [ ]:
mae = mean_absolute_error(b_test,B_pred)
mse = mean_squared_error(b_test, B_pred)
rmse = np.sqrt(mean_squared_error(b_test, B_pred))
r2 = r2_score(b_test, B_pred)
mae,mse,rmse,r2

(0.13062686288053277,
 0.02909411724022099,
 np.float64(0.17056997754652192),
 0.034832184263826504)

In [26]:
cato_test_1

,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Credit Score,Insurance Duration,Gender_Female,Gender_Male,Smoking Status_Yes,Smoking Status_No,Location_encoder
0,28.0,2310.0,4.0,7.657981,5.0,590.0,1.0,1.0,0.0,1.0,0.0,0


In [27]:
cato_test_1 = cato_test_1[sorted(cato_test_1.columns)]
    # Get the MLflow client
        #client = mlflow.tracking.MlflowClient()
model_name = "LinearRegression" 
        
scaler = MinMaxScaler()
col_1 =['Annual Income','Previous Claims','Predicted_Target_premium _account']


    
predicted_values = XGR_Boost.predict(cato_test_1)
cato_test_1["Predicted_Target_premium_account"] = predicted_values

le = LabelEncoder()
    
    # Fit label encoder using original training data categories
le.fit(cato_3['Location'])
    
    # Apply inverse transform
cato_test_1['Location_encoder'] = le.inverse_transform(cato_test_1['Location_encoder'])

col_2 = ['Gender','Smoking Status']
col_3 = ['Gender_Female','Gender_Male','Smoking Status_No','Smoking Status_Yes']
one = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
one.fit(cato_3[col_2])
cato_test_1[col_2] = one.inverse_transform(cato_test_1[col_3])
cato_test_3 = cato_test_1.drop(cato_test_1.columns[[6,7,8,9]], axis=1)


scaler.fit(cato_3[['Premium Amount']])
cato_test_3['Predicted_Target_premium_account'] = scaler.inverse_transform(cato_test_3[['Predicted_Target_premium_account']])

C:\Users\Hp\AppData\Local\Temp\ipykernel_8432\2635820780.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cato_test_1["Predicted_Target_premium_account"] = predicted_values
C:\Users\Hp\AppData\Local\Temp\ipykernel_8432\2635820780.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cato_test_1['Location_encoder'] = le.inverse_transform(cato_test_1['Location_encoder'])
C:\Users\Hp\AppData\Local\Temp\ipykernel_8432\2635820780.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [28]:
cato_test_3['Predicted_Target_premium_account']

0    1485.394409
Name: Predicted_Target_premium_account, dtype: float32

In [29]:
import streamlit as st
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import  mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np 
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
from cato import cato_1
from lable import label_1
from outlier import out_1
from min_max import mima
from data_cleaning import cleaning


st.title("Smart premium")

with st.form(key="data_form"):

  Age = st.text_input("Enter Your age")
  Number_of_Dependents = st.text_input("Enter Number of Dependents")
  Health_Score = st.text_input("Enter your Health Score")
  Credit_Score = st.text_input("Enter your Credit Score")
  Insurance_Duration = st.text_input("Enter your Insurance Duration")
  Gender = st.selectbox("Enter your Gender", ['Gender','Female', 'Male'])
  Smoking_Status = st.selectbox("Enter your Smoking Status", ['Smoking','No', 'Yes'])
  Location = st.selectbox("Enter your Location", ['Location','Urban','Rural','Suburban'])
  Annual_Income = st.text_input("Enter your Annual Income")
  Previous_Claims = st.text_input("Enter your Previous Claims")
  submitted = st.form_submit_button("Submit")

if submitted:
    data = {"Age" : [Age],"Number of Dependents":[Number_of_Dependents],"Health Score":[Health_Score],"Credit Score":[Credit_Score],"Insurance Duration":[Insurance_Duration],"Gender":[Gender],"Smoking Status":[Smoking_Status],"Location":[Location],"Annual Income":[Annual_Income],"Previous Claims":[Previous_Claims]}
    data_1 = pd.DataFrame(data)
    st.success("Wait for your premium amount")
    
    
    
    
    dx = pd.read_csv('C:/Users/Hp/Downloads/playground-series-s4e12/train.csv')
    dz = pd.read_csv('C:/Users/Hp/Downloads/playground-series-s4e12/test.csv')
    
    mlflow.set_experiment("XGR_Boost_Regressor_98")
    mlflow.set_tracking_uri("http://127.0.0.1:5000")
    with mlflow.start_run(run_name="MinmaxX"):
       
        train_data_1 = cleaning(dx)
        train_data = train_data_1.data_cleaning()


        cato_3 = train_data

        categorical_columns = ['Gender','Smoking Status']
        
        cato_train_99 = cato_1(cato_3,categorical_columns)
        cato_train = cato_train_99.categorical()

    
        cato_train_101 = label_1(cato_train)
        cato_train_1 = cato_train_101.label()

    
        train_out_90 = out_1(cato_train_1)
        train_out = train_out_90.Find_outlier()

    
        categorical_columns_cato_train_2=[]
        for cato_train_1_1 in train_out:
           categorical_columns_cato_train_2.append(cato_train_1_1)
            
        cato_train_38 = mima(cato_train_1,categorical_columns_cato_train_2)
        cato_train_3 = cato_train_38.min_max()

        A = cato_train_3.drop('Premium Amount', axis=1)
        b = cato_train_3['Premium Amount']
        A = A[sorted(A.columns)]

        test_data = data_1

        cato_test_10  = cato_1(data_1,categorical_columns)
        cato_test = cato_test_10.categorical()

    
        cato_test_101  = label_1(cato_test)
        cato_test_1 = cato_test_101.label()

    
        A_train, A_test, b_train, b_test = train_test_split(A, b, test_size = 0.2 , random_state = 42)
        


    
        XGR_Boost = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5,enable_categorical=True)
        XGR_Boost.fit(A_train,b_train)

        B_pred = XGR_Boost.predict(A_test)
        signature = infer_signature(A_train, XGR_Boost.predict(A_train))
        
        mae = mean_absolute_error(b_test,B_pred)
        mse = mean_squared_error(b_test, B_pred)
        rmse = np.sqrt(mean_squared_error(b_test, B_pred))
        r2 = r2_score(b_test, B_pred)
        name = "Minmax"
        model = XGR_Boost
    
        mlflow.log_param("Scaler", name)
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("MSE", mse)
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("R2 Score", r2)
        mlflow.sklearn.log_model(
        sk_model=XGR_Boost,
        artifact_path="sklearn-model",
        signature=signature,
        registered_model_name="XGR_Boost",
    )
        
       
        cato_test_1 = cato_test_1[sorted(cato_test_1.columns)]
    # Get the MLflow client
        #client = mlflow.tracking.MlflowClient()
        model_name = "XGR_Boost" 
        
        scaler = MinMaxScaler()
        col_1 =['Annual Income','Previous Claims','Predicted_Target_premium _account']


    
        predicted_values = XGR_Boost.predict(cato_test_1)
        cato_test_1["Predicted_Target_premium_account"] = predicted_values

        le = LabelEncoder()
    
    # Fit label encoder using original training data categories
        le.fit(cato_3['Location'])
    
    # Apply inverse transform
        cato_test_1['Location_encoder'] = le.inverse_transform(cato_test_1['Location_encoder'])

        col_2 = ['Gender','Smoking Status']
        col_3 = ['Gender_Female','Gender_Male','Smoking Status_No','Smoking Status_Yes']
        one = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
        one.fit(cato_3[col_2])
        cato_test_1[col_2] = one.inverse_transform(cato_test_1[col_3])
        cato_test_3 = cato_test_1.drop(cato_test_1.columns[[6,7,8,9]], axis=1)


        scaler.fit(cato_3[['Premium Amount']])
        cato_test_3['Predicted_Target_premium_account'] = scaler.inverse_transform(cato_test_3[['Predicted_Target_premium_account']])
        
    mlflow.end_run()


    st.write(cato_test_3['Predicted_Target_premium_account'])



2025-04-21 16:09:55.767 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 16:09:56.737 
  command:

    streamlit run c:\Users\Hp\OneDrive\Desktop\python\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-04-21 16:09:56.747 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 16:09:56.756 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 16:09:56.759 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 16:09:56.760 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 16:09:56.762 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-21 16:09:56.764 Thread 'Main

In [30]:
cato_test_1

,Age,Annual Income,Credit Score,Gender_Female,Gender_Male,Health Score,Insurance Duration,Location_encoder,Number of Dependents,Previous Claims,Smoking Status_No,Smoking Status_Yes,Predicted_Target_premium_account,Gender,Smoking Status
0,28.0,2310.0,590.0,1.0,0.0,7.657981,1.0,Rural,4.0,5.0,0.0,1.0,0.294315,Female,Yes
